In [1]:
!pip install anthropic

   ---------------------------------------- 0.0/75.6 kB ? eta -:--:--
   ---------------------------------------- 75.6/75.6 kB 4.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/77.9 kB ? eta -:--:--
   ---------------------------------------- 77.9/77.9 kB 4.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/58.3 kB ? eta -:--:--
   ---------------------------------------- 58.3/58.3 kB 3.2 MB/s eta 0:00:00
  Attempting uninstall: h11
    Found existing installation: h11 0.9.0
    Uninstalling h11-0.9.0:
      Successfully uninstalled h11-0.9.0
  Attempting uninstall: httpcore
    Found existing installation: httpcore 0.9.1
    Uninstalling httpcore-0.9.1:
      Successfully uninstalled httpcore-0.9.1
  Attempting uninstall: httpx
    Found existing installation: httpx 0.13.3
    Uninstalling httpx-0.13.3:
      Successfully uninstalled httpx-0.13.3


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
googletrans 4.0.0rc1 requires httpx==0.13.3, but you have httpx 0.27.0 which is incompatible.


In [1]:
import pandas as pd

file_path = "./prepro_company_data.csv"
df = pd.read_csv(file_path)
df["Positive_Score"] = None
df

,company,code,Date,Title,Positive_Score
0,삼성전자,'005930,2021-03-12,진짜 5 G 28 서비스 활성화 첫 발 과기부 TF 발족 회의,None
1,삼성전자,'005930,2021-03-05,아버님 댁 에어 프라이어 놓다 드리다,None
2,삼성전자,'005930,2021-05-14,ESG 경영 첫 단추 늘다 여성 관리자 키우다,None
3,삼성전자,'005930,2021-02-05,삼성 공장 증설 19조 투자 주 정부 9000억 세제 요청,None
4,삼성전자,'005930,2021-01-22,화웨이 서다 분리 중 가폰 어너 인텔 삼성 반도체 계약,None
...,...,...,...,...,...
15895,코오롱인더,'120110,2023-07-21,연구개발 1.1조 쏟다 코오롱 인 더 하반기 과실 딴다,None
15896,코오롱인더,'120110,2023-10-20,코오롱 인 더 3 Q 영업 익 시장 기 대치 회 4 Q 패션 성 수기 효과 기대 IBK,None
15897,코오롱인더,'120110,2023-10-27,코오롱 몰다 파격 할인 2023 OLO 모드 K 쇼핑 위 크다 진행,None
15898,코오롱인더,'120110,2023-12-01,추다 날씨 아우 터 판매 확대 코오롱 인 더 실적 개선 전망 IBK,None


In [1]:
import anthropic
import pandas as pd
import re
import os

# CSV 파일 경로
file_path = "./Claude.csv"

# 파일 존재 여부 확인
if os.path.exists(file_path):
    df = pd.read_csv(file_path)
else:
    df.to_csv(file_path, index=False, encoding='utf-8-sig')  # index를 False로 설정하여 인덱스를 저장하지 않음

api_key = "your_key"

# Anthropic 클라이언트 설정
client = anthropic.Anthropic(api_key=api_key)

model = "claude-3-haiku-20240307"

import time

# 감성 분석 및 점수 저장
while True:
    try:
        for idx, row in df.iterrows():
            title = row['Title']
            company = row['company']
            # 'Positive_Score' 값이 이미 있는 경우 건너뛰기
            if pd.notnull(row['Positive_Score']):
                continue
            print(f"Processing headline {idx}: {title}")
            user_prompt = f"{title} .좋은 뉴스, 나쁜 뉴스, 불확실한 뉴스 세 가지가 될 확률이 합쳐서 1일 때 좋은 뉴스일 정도를 확률로만 답변해주세요. (예시답변: 0.2) {company}에 좋은 영향을 숫자로만 답해주세요"
            response = client.messages.create(
                model=model,
                max_tokens=1000,
                temperature=0,
                system="금융전문가로 가정해주세요. 당신은 주식 추천 경험이 있는 금융 전문가 입니다.",
                messages=[
                    {"role": "user", "content": user_prompt}
                ]
            )
            # 응답에서 숫자만 추출하여 저장
            score = re.findall(r'\d+\.\d+', response.content[0].text)
            if score:
                df.at[idx, 'Positive_Score'] = float(score[0])
                df.to_csv(file_path, index=False, encoding='utf-8-sig')
            else:
                print(f"No score found for headline {idx}")
        break  # 에러 없이 완료되었으면 반복문 종료
    except Exception as e:
        print(f"An error occurred.{str(e)}  Retrying in 1 minute...")
        time.sleep(63)  # 1분 대기 후 다시 시도

print("감성 분석이 완료되었습니다.")

감성 분석이 완료되었습니다.


In [9]:
import pandas as pd

file_path = "./prepro_company_data.csv"
df = pd.read_csv(file_path)
df["Positive_Score"] = None
df

,company,code,Date,Title,Positive_Score
0,삼성전자,'005930,2021-03-12,진짜 5 G 28 서비스 활성화 첫 발 과기부 TF 발족 회의,None
1,삼성전자,'005930,2021-03-05,아버님 댁 에어 프라이어 놓다 드리다,None
2,삼성전자,'005930,2021-05-14,ESG 경영 첫 단추 늘다 여성 관리자 키우다,None
3,삼성전자,'005930,2021-02-05,삼성 공장 증설 19조 투자 주 정부 9000억 세제 요청,None
4,삼성전자,'005930,2021-01-22,화웨이 서다 분리 중 가폰 어너 인텔 삼성 반도체 계약,None
...,...,...,...,...,...
15895,코오롱인더,'120110,2023-07-21,연구개발 1.1조 쏟다 코오롱 인 더 하반기 과실 딴다,None
15896,코오롱인더,'120110,2023-10-20,코오롱 인 더 3 Q 영업 익 시장 기 대치 회 4 Q 패션 성 수기 효과 기대 IBK,None
15897,코오롱인더,'120110,2023-10-27,코오롱 몰다 파격 할인 2023 OLO 모드 K 쇼핑 위 크다 진행,None
15898,코오롱인더,'120110,2023-12-01,추다 날씨 아우 터 판매 확대 코오롱 인 더 실적 개선 전망 IBK,None


In [2]:
import anthropic
import pandas as pd
import re
import os

# CSV 파일 경로
file_path = "./Claude_sonnet.csv"

# 파일 존재 여부 확인
if os.path.exists(file_path):
    df = pd.read_csv(file_path)
else:
    df.to_csv(file_path, index=False, encoding='utf-8-sig')  # index를 False로 설정하여 인덱스를 저장하지 않음

api_key = ""

# Anthropic 클라이언트 설정
client = anthropic.Anthropic(api_key=api_key)

model = "claude-3-sonnet-20240229"

import time

# 감성 분석 및 점수 저장
while True:
    try:
        for idx, row in df.iterrows():
            title = row['Title']
            company = row['company']
            # 'Positive_Score' 값이 이미 있는 경우 건너뛰기
            if pd.notnull(row['Positive_Score']):
                continue
            print(f"Processing headline {idx}: {title}")
            user_prompt = f"{title} .좋은 뉴스, 나쁜 뉴스, 불확실한 뉴스 세 가지가 될 확률이 합쳐서 1일 때 좋은 뉴스일 정도를 확률로만 답변해주세요. (예시답변: 0.2) {company}에 좋은 영향을 숫자로만 답해주세요"
            response = client.messages.create(
                model=model,
                max_tokens=1000,
                temperature=0,
                system="금융전문가로 가정해주세요. 당신은 주식 추천 경험이 있는 금융 전문가 입니다.",
                messages=[
                    {"role": "user", "content": user_prompt}
                ]
            )
            # 응답에서 숫자만 추출하여 저장
            score = re.findall(r'\d+\.\d+', response.content[0].text)
            if score:
                df.at[idx, 'Positive_Score'] = float(score[0])
                df.to_csv(file_path, index=False, encoding='utf-8-sig')
                time.sleep(2)
            else:
                print(f"No score found for headline {idx}")
        break  # 에러 없이 완료되었으면 반복문 종료
    except Exception as e:
        print(f"An error occurred.{str(e)}  Retrying in 1 minute...")
        time.sleep(63)  # 1분 대기 후 다시 시도

print("감성 분석이 완료되었습니다.")

Processing headline 5825: 되다 살아나다 투심 2 차다 날다
No score found for headline 5825
Processing headline 10161: 챗 GPT 열풍 AIIT 반도체 관련 주 들썩
Processing headline 10162: 인터넷 뱅크 전환 설 부인 에도 주가 쭉쭉 초고 수도 베팅 하다 종목
Processing headline 10163: 주식 초 고수 늘다 지금 과도 하다 프리미엄 지적 에코 프로 순 매수 1 위 코스피 2 ...
Processing headline 10164: 주식 초 고수 늘다 지금 AI 반도체 수혜 기 대다 이수 페타 시스 순 매수 1 위
Processing headline 10165: 주식 초 고수 늘다 지금 코스피 이전 상장 검토 엘앤 에프 순 매수 1 위
Processing headline 10166: 에코 프로 황 제주 탈환 코스닥 3%
Processing headline 10167: MSCI 지수 리밸런싱 실시 9월 수급 기 대감
Processing headline 10168: 희비 갈리다 반도체 2 차다 잭슨 홀 앞두다 눈치 싸움
Processing headline 10169: 올해 증시 달구다 상위 20 개 종목 은 1 위 에코 프로 2 위 제이 엘 케이
Processing headline 10170: 시 총 1조달러 무너지다 엔비디아 반도체
Processing headline 10171: 국민연금 2 차다 유통 줄 이고 중국 수혜 주소 부장 더 담다
Processing headline 10172: 삼성 vs 미래에셋 AI 소 부장 ETF 한판 승부
Processing headline 10173: 미래에셋 TIGER AI 반도체 핵심 공정 ETF 상장 3일 만에 개인 순 매수 100 ...
Processing headline 10174: 경제 계 인사 네이버 최고 운영 책임자 김범준
Processing headline 10175: 구글 미나 개발 소식 AI 관련 주 

Processing headline 10282: 해외 공장 늘리다 K 전선 해상 풍력 바람 타고 4조원 벽 넘다
Processing headline 10283: 호재 들썩이다 금천구 랜드마크 신 독산역 월드메르디앙 센트 원
Processing headline 10284: 대한전선 기업 지배구조 헌장 공표 ESG 경영 강화 박차
Processing headline 10285: 러시아 원자재 수출 금지 품목 추측 난무 ... 요동치다 원자재 관련 주
Processing headline 10286: 윤 태식 관세청장 LA 현지 진출 기업 간담 회 개최
Processing headline 10287: 똑같다 코스피 아니다 수익률 더 높다 동 일가 중 ETF 주목
Processing headline 10288: 대한전선 영업 이익 확대 되다 흑자 전환 매출 전 대비 41% 증가
Processing headline 10289: 특징 주 대한전선 640조 옴 시티 회 동 기 대감 속 사우디 현지 합작 설립 ...
Processing headline 10290: 산업 은행 한국 GM 지다 분 17% 매각 추진 시기 늘다 미정
Processing headline 10291: 호반 건설 7일 경력 공개 채용 마감
Processing headline 10292: 대한전선 ESG 평가 2년 연속 통합 A 등급 획득
An error occurred.Error code: 529 - {'type': 'error', 'error': {'type': 'overloaded_error', 'message': 'Overloaded'}}  Retrying in 1 minute...
Processing headline 5825: 되다 살아나다 투심 2 차다 날다
No score found for headline 5825
Processing headline 10292: 대한전선 ESG 평가 2년 연속 통합 A 등급 획득
Processing headline 10293: 풍력 태양광 수요 늘다 K 전선 

Processing headline 10376: 11월 부울경 상 장사 주가 10 곳 중 9 곳 하락
Processing headline 10377: 임금 부담 가중 줄 소송 위기 신의칙 법적 지침 마련
Processing headline 10378: 탄소 중립 시대 필수 탄소 운반 선 개발 하다 글로벌 조선업 계
Processing headline 10379: 현대 중공업 대우조선 합병 불발 에도 연초 수주 잭팟
Processing headline 10380: 합병 무산 희비 교차 한조 양 구름 걷히다 vs 대조 양 먹구름
Processing headline 10381: 구조조정 버티다 현 대미 포 .. 중형 선박 강 부상 오다 목표 140% 달성
Processing headline 10382: 조선업 패러다임 전환 한국 조선해 양 백 년대 계 키우다 데스크 칼럼
Processing headline 10383: 조선업 불확실 성 이미 주가 반영 1분 기 회복 예상
Processing headline 10384: 르포 그린 쉬다 칼 디지털 전환 은 방패 스마트 조선소 구축 나선 한국 조선해 양
Processing headline 10385: 현 대미 포 조선 인천 제주 뱃길 잇다 카페리선 건조
Processing headline 10386: 현대 지난해 임금 협상 난항 설 이후 교섭 서도 합의 실패
Processing headline 10387: 성공예감 조선업 주가 늘다 OO 제일 중요하다 대신 증권 자산 리서치 부 이동헌 팀 ...
Processing headline 10388: 한국 조선해 양 LPG 선 컨테이너선 5 척 2900억원 수주
Processing headline 10389: 조 선방 산 중후 장대 업계 불다 이사 열풍
Processing headline 10390: 하다 영석 현대 부회장 글로벌 품질 경영인 대상 수상
Processing headline 10391: LNG 가다 컨테이너선 수소 전지 선박 현대 친환경 앞장
Processing 

Processing headline 10515: 클릭 e 종목 FF 견조 하다 실적 개선 상보
Processing headline 10516: CNN 빌보드 UFC 폭스바겐 이종 교배 라이선스 패션 브랜드 봇물
Processing headline 10517: 코스피 찬 바람 에도 따뜻하다 의류 주 ... 하다 세 실업 원 무역 리 오프닝 훈풍
Processing headline 10518: FF 드라마 제작사 빅토리 콘텐츠 인수 시너지 기대
Processing headline 10519: 인플레 국채 금리 급등 코스피 하락 마감
Processing headline 10520: 경기 둔화 우려 코스피 코스닥 동반 하락
Processing headline 10521: 버추 얼 콘텐츠 플랫폼 몰팅 엔슬파트너스 로 시드 투자 유치
Processing headline 10522: 단독 원면 가격 역대 최고 치 자라 물다 본격 인상
Processing headline 10523: NH 섬유 의복 산업 비중 확대 추천 수 혜주 FF 영원 무역
Processing headline 10524: 급락 장 속 구원투수 판하 돌아오다 연기금 일주일 만에 사자
Processing headline 10525: 하다 5만 전자 가 진짜 경 기 침체 대비 라
Processing headline 10526: 수십 만원 짜다 티셔츠 도 불티 제대로 대박 터지다 사업 박동 휘 컨슈머 리포트
Processing headline 10527: FF 골프 콘텐츠 투자 이다 테니스 브랜드 인수
Processing headline 10528: 국내 의류 판매 성장 세 지속 패션 업체 주가 하락 과도 하다 신한
Processing headline 10529: 신흥 럭셔리 뒤지다 놓다 골프 웨어 시장 엔 데믹 앞두다 누가 살아나다 ...
Processing headline 10530: 데이터 보다 증시 코스피 기관 외국인 개인 순 수 순 매도 상위 종목 11월 4일 ...
Processing headline

Processing headline 10653: 코오롱 모빌리티 BMW 분할 숨다 속 내 VaultMarket
Processing headline 10654: 중국 서다 자다 나가다 ... 휠라 늘다 뜨다 FF 늘다 지다 이유
Processing headline 10655: 혼돈 증시 외국인 연속 사자 종목 들여다보다
Processing headline 10656: 휠라 홀딩스 3 Q 실적 기 대치 회 중립 유지 NH
Processing headline 10657: 아쿠 쉬 네트 날다 재고 부담 휠라 홀딩스 수익 성
Processing headline 10658: 남 은 월급 번 다는데 천 피 시대 일개미 허탈
Processing headline 10659: 아모레 G 자회사 퍼시픽 글라스 지다 분 프랑스 베르상스 매각
Processing headline 10660: 마스크 벗다 화장 하다 수 있다 다시 뛰다 아모레퍼시픽 그룹 포트 라이트
Processing headline 10661: 아모레 장녀 서민정 보광 창투 장남 홍 정환 합의 이혼 종합
Processing headline 10662: 잘나가다 탈 마스크 델타 변 화들짝 ... 위축 되다 투심
Processing headline 10663: 히트 상품 처럼 광고 하다 ... ETF 수익률 은 온통 마이너스
Processing headline 10664: 매출 4년 만에 반토막 C 뷰티 밀리다 이니스프리 아프다 손가락 돼다
Processing headline 10665: 고전 아모레 G 3분 기 부진 연말 보복 소비 실적 회복 노리다
Processing headline 10666: 역병 딛다 돌아오다 설화수 아모레 그녀 이끌다 K 뷰티
Processing headline 10667: 럭셔리 브랜드 온라인 양 날개 K 뷰티 올해 다시 날아오르다
Processing headline 10668: 마스크 호재 어닝 쇼크 덮다 아모레퍼시픽 고공 행진
Processing headline 10669: 빅데이터 MSI

...

Processing headline 15311: 인플레 대안 이르다 더니 달 만에 31% 급락 하다 상품
Processing headline 15312: 금리 인상 부담 덜다 리츠 인프라 펀드 눈 띄다 WEALTH
Processing headline 15313: 크다 드럽다 플 크라운 달성 하다 김성현 KB 대표 블라인드 펀드 결성 올해 늘다 MA 주력
Processing headline 15314: 킹 달러 끝 연말 물가 2% 대다 달러 투자 전문가 주목 하다 은
Processing headline 15315: 투 자금 선제 확보 반도체 업계 실탄 확보 안간힘
Processing headline 15316: 리츠 지다 분 늘리다 미래에셋 운용 투자 적기
Processing headline 15317: 외국인 지갑 연 다 꿈틀 대다 늘다 리츠
Processing headline 15318: 시그널 미래에셋 이지스 운용 리츠 배당률 치솟다 줍다 줍다
Processing headline 15319: 거래소 넥 스틸 상장 예심 승인 ... 올해 첫 코스피 IPO
Processing headline 15320: 금리 상승 막바지 리츠 저가 매수 기 대감 쑥
Processing headline 15321: CCIM 한국 협회 상장 리츠 A to Z 세미나 24일 개최
Processing headline 15322: SK 리츠 수처리 센터 잠재 증자 물량 줄이다 안간힘
Processing headline 15323: SK 팔리다 반년 만에 440억 껑충 사옥 뭐길 래
Processing headline 15324: 넘버스 SK 하이닉스 차 입금 30조 에도 안도 하다 수 있다 이유
Processing headline 15325: 신주인수권 단돈 1원 SK 리츠 유상증자 잡음 주주 외면
Processing headline 15326: 하이닉스 수처리 시설 1조 사고 SK 늘다 유증 사실 상 불참 ... SK 리츠 주주 ...
Processing headline 15327:

Processing headline 15449: 가구 업계 하반기 기지개 완만 하다 실적 상승세
Processing headline 15450: 하다 샘 3분 기 영업 익 49억원 흑자 전환
Processing headline 15451: 하다 샘 삼성 스마트 싱스 연동 신제품 9 종 출시
Processing headline 15452: 알루미늄 가격 급등 관련 주 출렁 삼다 알미늄 15%
Processing headline 15453: 특징 주 삼다 알미늄 최재형 대선 출마 회장 대표이사 학연 부각
Processing headline 15454: 특징 주 삼다 알미늄 최재형 국민 힘 입당 소식 6 거래 연속 상승세
Processing headline 15455: 알루미늄 슈퍼 사이클 올라타다
Processing headline 15456: 알루미늄 연 철강 이름 값 늘다 원자재 관련 주
Processing headline 15457: 공급 줄다 수요 늘다 쑥 알루미늄 주 들썩들썩
Processing headline 15458: 달아오르다 철강 알루미늄 더 달리다
Processing headline 15459: 7월 미디어 엔 터 테마 강세 최고 상 승률 종목 은 에스티 큐브
Processing headline 15460: 연휴 앞두다 코스피 코스닥 기분 좋다 마감
Processing headline 15461: 배터리 빅 3 손잡다 삼다 알미늄
Processing headline 15462: JKL 파트너 스 모빌리티 펀드 크다 징 임박 6 호 블라인드 시동
Processing headline 15463: 특징 주 삼다 알미늄 LG 엔솔 손잡다 미국 진출 소식 15% 강세
Processing headline 15464: 관세청 납세 날 모범 납세 포상 정부 포상 수상 8 명
Processing headline 15465: 동전주 빼다 이차전지 도 하락 전환 거품 꺼지다
Processing headline 15466: 거래 액 2000 배 급증 2 차다 단 타

Processing headline 15586: GTX 역다 권 오피스텔 e 편 한세상 시티 부평역 14일 분양 홍보 관 개관
Processing headline 15587: 대림 피앤피 환경 공 단 전북도청 농협 영농 폐 비닐 재활용 MOU 체결
Processing headline 15588: 몸값 6조 현대 엔지니어링 건설 대장 예고
Processing headline 15589: DL 건설 안전 보건 경영 시스템 인증 취득
Processing headline 15590: LX 인터내셔널 바이오매스 발전소 인수
Processing headline 15591: 수도권 서다 희망고문 당하다 청포 족 향 하다 곳 은
Processing headline 15592: DL 건설 협력 업체 계약 보증 수수료 환급 동반성 장 추구
Processing headline 15593: 한국 맥도날드 너마저 프랜차이즈 줄줄이 MA 매물 나오다 이유
Processing headline 15594: e 편 한세상 시티 청라 9월 분양 청라 첫 e 편 한세상 시티 상륙
Processing headline 15595: 하다 경 라씨 수주 공시 반응 일진 홀딩스 테크 윙 DL 건설
Processing headline 15596: DL 건설 일산 12 구역 재개발 수주 정비 사업 누적 1조원 육박
Processing headline 15597: 2분 기 성장 률 0.3% p 상향 조정
Processing headline 15598: 대보 건설 김원태 신임 대표이사 선임
Processing headline 15599: 주간 분양 전매 제한 완화 전국 1176 가구 청약 접수
Processing headline 15600: DL 건설 한국 ESG 기준 원 평가 서다 종합 A 등급
Processing headline 15601: 주간 분양 분양 시장 활기 찾다 4190 가구 청약 접수
Processing headline 15602: 패션 커머스 브랜디 7 개월 만에 거래 액 400억 늘리다 비

Processing headline 15724: 투자 기대 보다 분할 우려 ... 힘 못 쓰다 SK 그룹 주
Processing headline 15725: 개인 공매도 늘다 쥐꼬리
Processing headline 15726: 특징 주 대정 화금 SK 바사 3 상 승인 관련 주 수혜 기 대감 강세
Processing headline 15727: 카뱅 청약 흥행 기대 주주 몸값 도 급등
Processing headline 15728: 파마 리서치 부작용 없다 관절 강 주사 앞세우다 고 성장
Processing headline 15729: 주식 초 고수 늘다 지금 여전하다 배터리 소재 기 대감 ... 엘앤 에프 에코 프로비 엠 사고 ...
Processing headline 15730: 페트병 운동화 인기 가치 소비 업다 자원 선 순환 노리다
Processing headline 15731: 최재원 복귀 인사 실험 북미 강화 촉각
Processing headline 15732: CJ 제일제당 HDC 현대 EP 화이트 바이오 합작 법인 설립
Processing headline 15733: 관가 뒷 담 SK 실 트론 사익 편취 히든 플레이어 늘다 공정위 OB
Processing headline 15734: 해외 에서는 금지 되다 쪼개다 상장 기업 물 적 분할 개미 은 피 눈물
Processing headline 15735: 시위 국민 청원 물 적 분할 개미 뿔 나다
Processing headline 15736: 임 상 성과 속 속 테 고 사이언스 최대성 기대
Processing headline 15737: 주 개발 공 사한국 도로공사 전국 휴게소 투명 페트병 순환 체계 구축
Processing headline 15738: 휴비스 엠셀 투자 계약 체결 스마트 섬유 시장 개척
Processing headline 15739: AI 신약 개발 업계 자체 합성 연구소 신설 공 들이다 이유
Processing headline 15740: SK 케미칼 2025년 매출 4조원 

Processing headline 15858: 바디 프 랜드 2023 코리아 세 일 페스 타 출격 안마 의자 최대 50% 할인
Processing headline 15859: 공항 KTX 역 필요하다 그린 카 클린 존 경험 하다
Processing headline 15860: 코스피 마감 외인 이다 기관 팔자 2450 선 뚝
Processing headline 15861: 코스닥 마감 기관 외인 거 세다 도세 하락 1051 선
Processing headline 15862: 없다 수 소 전기차 도 없다 원조 베다 처 일진 자신감
Processing headline 15863: 유럽 가다 일진 그룹 친환경 탄소 전략 매진 하다 이유
Processing headline 15864: 일진 그룹 기획 재무 하반기 신입 경력 공채
Processing headline 15865: 일진 그룹 연구개발 허브 이노 센터 개관
Processing headline 15866: 재산 공개 김주현 금융 위원장 29억원 신성환 금통위 원 45억원
Processing headline 15867: 전남도 에너지 신 산업 도약 글로벌 혁신 특구 유치하다
Processing headline 15868: 일진 전기 초 고압 변압기 공장 682억원 증설 투자
Processing headline 15869: 일진 전기 1000억원 상당 유상증자 결정 대규모 시설 자금 투입
Processing headline 15870: 일진 그룹 6 개사 2023년 하반기 공채
Processing headline 15871: 코오롱 그룹 사업 성공 공유 하다 코오롱 페스티벌 개최
Processing headline 15872: 특징 주 어닝 서프라이즈 코오롱 인 더 3% 대다 급등
Processing headline 15873: 코오롱 인 더 지역 사회 상생 하다 ESG 실천
Processing headline 15874: 클릭 e 종목 코오롱 인 더 단 기중기 장기 기대 해보다 해
Processing head

In [1]:
import anthropic
import pandas as pd
import re
import os

# CSV 파일 경로
file_path = "./Claude_opus.csv"

# 파일 존재 여부 확인
if os.path.exists(file_path):
    df = pd.read_csv(file_path)
else:
    df.to_csv(file_path, index=False, encoding='utf-8-sig')  # index를 False로 설정하여 인덱스를 저장하지 않음

api_key = ""

# Anthropic 클라이언트 설정
client = anthropic.Anthropic(api_key=api_key)

model = "claude-3-opus-20240229"

import time

# 감성 분석 및 점수 저장
while True:
    try:
        for idx, row in df.iterrows():
            title = row['Title']
            company = row['company']
            # 'Positive_Score' 값이 이미 있는 경우 건너뛰기
            if pd.notnull(row['Positive_Score']):
                continue
            print(f"Processing headline {idx}: {title}")
            user_prompt = f"{title} .좋은 뉴스, 나쁜 뉴스, 불확실한 뉴스 세 가지가 될 확률이 합쳐서 1일 때 좋은 뉴스일 정도를 확률로만 답변해주세요. (예시답변: 0.2) {company}에 좋은 영향을 숫자로만 답해주세요"
            response = client.messages.create(
                model=model,
                max_tokens=1000,
                temperature=0,
                system="금융전문가로 가정해주세요. 당신은 주식 추천 경험이 있는 금융 전문가 입니다.",
                messages=[
                    {"role": "user", "content": user_prompt}
                ]
            )
            # 응답에서 숫자만 추출하여 저장
            score = re.findall(r'\d+\.\d+', response.content[0].text)
            if score:
                df.at[idx, 'Positive_Score'] = float(score[0])
                df.to_csv(file_path, index=False, encoding='utf-8-sig')
            else:
                print(f"No score found for headline {idx}")
        break  # 에러 없이 완료되었으면 반복문 종료
    except Exception as e:
        print(f"An error occurred.{str(e)}  Retrying in 1 minute...")
        time.sleep(63)  # 1분 대기 후 다시 시도

print("감성 분석이 완료되었습니다.")

Processing headline 10194: 상품 개발 대상 손해 보험 협회장 상 현대 해상 화재보험 유 병자 도 건강 보험 혜택 ... 3 대다 ...
Processing headline 10195: 마인 즈 랩 증권 신고 서다 제출 ... 내달 코스닥 상장
Processing headline 10196: 마켓 인 마인 즈 랩 일반 경쟁률 90 대다 1 증거 금 2039억
Processing headline 10197: 손보 빅 4 지난해 3조원 벌다 역대 최대
Processing headline 10198: 보험사 약관 대출 증가 주 담대 재개 풍선 효과 다시 시작 되다
Processing headline 10199: 삼성 한화 교보 RBC 하락 금리 인상 건전성 관리 비상 종합
Processing headline 10200: 모르다 손해 보험 가입 안해 도 보험금 받다 수 슬기롭다 금융 생활
Processing headline 10201: 현대 해상 지난해 순 익 30% 뛰다 1480원 현금 배당
Processing headline 10202: 보험 업계 건보 공단 데이터 사용 심의 연기
Processing headline 10203: 손 보사 순 익 크게 늘다 덩치 3 배 생 보사 바짝 추격
Processing headline 10204: 동학 개미 악몽 상반기 코스피 21% 급락 삼전 카카오 신저
Processing headline 10205: 마켓 인 TS 인베 퓨처 플레이 포기 투 인슈 테크 오픈 플랜 30억원 투자
Processing headline 10206: 손 보사 빅 5 1분 기 순 익 1조 넘다
Processing headline 10207: 금리 인상 유일하다 수 혜주 늘다 보험 주 ... 빅스 텝 발언 보험 선방
Processing headline 10208: 현대 해상 정몽윤 회장 재 선임 3년 임기
Processing headline 10209: 1 mm 금융 톡 마음 케어 어린이 보험 관심 급증 하다 이유
Processing

Processing headline 10329: 미성년 최고 주식 부호 800억 대다 남매 ... 증가 율 톱 은 학원 재벌
Processing headline 10330: 한미 약품 사이언스 MSD 먹다 코로나 치료 복 제약 생산 강세
Processing headline 10331: 한미 약품 월경 전 느끼다 불편하다 진 보라 지유 케어 하다
Processing headline 10332: 한미 약품 영국 자동 조 제기 시장 진출 JVM 해외 사업 속도
Processing headline 10333: 승계 구도 신약 FDA 승인 다시 출발 하다 한미 약품
Processing headline 10334: 기업 탐사 롯데 관광 개발 최대 주주 지다 분 대부분 담보 넣다 돈 빌리다
Processing headline 10335: 계 열사 뭉치다 약 바이오 불다 합병 바람
Processing headline 10336: 이브 이엠 3분 기 누적 매출 1000억 돌파
Processing headline 10337: 미래 먹거리 직접 찾다 키우다 스타트업 육성 나선 중견 약사
Processing headline 10338: 블루 오션 되다 디지털 치료 국내 1 호 주인공 은
Processing headline 10339: 신다 카바 이오 한미 계약 유지 주장 ... 불확실 성 UP
Processing headline 10340: JVM 지난해 매출 1419억 영업 이익 219억 전 22.6% 76% ...
Processing headline 10341: 역대 최대 실적 상장 약사 화끈하다 현금 배당
Processing headline 10342: 넥슨 도 한미 약품 식 모델 활용 하다 ... 상속 시장 뛰어들다 PEF
Processing headline 10343: 새마을금고 라데팡스 펀드 출자 결정 한숨 돌리다 한미 약품
Processing headline 10344: 가뭄 콩 제약업 계 여성 CEO 마다 대부분 은 가족 경영
Processing headline 10

Processing headline 10468: 빈대 인 신임 회장 진심 통 하다 BNK 공정하다 인사 문화 정착 시키다
Processing headline 10469: BNK 부산 은행 원 도심 학생 2600 명 수능 온라인 강좌 지원
Processing headline 10470: DS 투자 증권 BNK 금융 지주 돋보이다 주주 가치 고 의지
Processing headline 10471: 파리 게이츠 핑 믿다 200억 CB 투자 하다 주가 40% 급락 금융 물리다
Processing headline 10472: 지방 리딩 뱅크 나야 DGBJB 최대 실적 에도 부진 BNK 1 위 수 ...
Processing headline 10473: 일문일답 김주현 금융 위원장 새마을금고 건전성 문제 없다 DSR 안 풀다
Processing headline 10474: BNK 금융 경남 은행 횡령 사고 관련 내부통제 혁신 총괄 컨트롤 타워 운영 중
Processing headline 10475: 빈대 인 BNK 금융 회장 횡령 사고 유감 내부통제 개선
Processing headline 10476: BNK 부산 경남 은행 장기 근무 70 명 인사 단행
Processing headline 10477: 박스 피 지루하다 외인 먼저 몰리다 가다 배당 주 TOP 3
Processing headline 10478: 배당 수익률 10% 육박 외국인 기관 쌍끌이 사들이다
Processing headline 10479: 지방 금융 지주 향 하다 칼날 금감원 내부통제 점검 나서다
Processing headline 10480: 경남 거액 횡령 사고 SP BNK 금융 내부통제 리스크 확대
Processing headline 10481: 블록 체인 위 크다 인 부산 2023 11월 9일 개최
Processing headline 10482: BNK 투자 증권 새 사장 신명호
Processing headline 10483: FF 중국 글로벌 시장 매출 목표 주가 일제 히 포트 라이트
Proc

Processing headline 10606: 경남 이다 대다 전북 금감원 지방 금융 지주 칼 겨눈 다
Processing headline 10607: 오다 뉴 DGB 최대 주주 지다 분 하다 끗 차이 OK 저축은행 최종 수혜자 되다
Processing headline 10608: JB 금융 지주 사외이사 후보자 주주 추천 접수
Processing headline 10609: 금융 주 배당금 배 당일 도입 분산 투자 전략 수정 필요
Processing headline 10610: BNK 금융 15년 간 성 장주 밀리다 주가 재 평가 확신
Processing headline 10611: AI 뉴스 공시 분석 해 ESG 점수 산출 투자자 도움 되다 겁니다
Processing headline 10612: 금융위 ESG 공시 기준 내년 1분 기 중 구체화 기후 검토
Processing headline 10613: 당국 불호령 역대 최대 실적 은행 주 흔들다
Processing headline 10614: 윤 윤수 휠라 코리아 회장 지난해 보수 23.6억원 수령
Processing headline 10615: 금투 휠라 홀딩스 1분 기 실적 모멘텀 기대
Processing headline 10616: 환경 앞장서다 휠라 ESG 성과 집약 하다 지속 가능 보고서 발간
Processing headline 10617: 골프 DNA 진해 지다 휠라 타이틀 리스트 휠라 골프 이끌다
Processing headline 10618: 종목 속 골프 치다 BTS 운동화 신다 매력 적 인 휠라 홀딩스
Processing headline 10619: 특징 주 휠라 홀딩스 자회사 깜짝 실적 강세 6%
Processing headline 10620: K 패션 샵 오픈 힙 해진 호주 멜버른 ... K 패션 이유 있다 인기
Processing headline 10621: 자금 조달 휠라 홀딩스 아쿠 쉬 네트 보유 매그너스 증자 용 800억 모채 발행
Processing headline 1062

Processing headline 10743: 에스원 국내 5 대다 브랜드 서비스 품질 인증 1 위
Processing headline 10744: 휴가 철 빈집 털 많다 늘다 7월 4 주차 심야시간
Processing headline 10745: 에스원 종합 부동산 서비스 새롭다 패러다임 제시
Processing headline 10746: 성장하다 가치주 투자 하다 퀄리티 펀드 자금 유입 인기
Processing headline 10747: 에스원 ESG 경영 활동 시동 2050년 탄소 배출 제로 화
Processing headline 10748: 더 편리하다 에스원 비대 면 서비스
Processing headline 10749: 에스원 보안 업계 최초 KCGS ESG A 등급 획득
Processing headline 10750: 경제 계 인사 투자 증권 사장 홍원식 씨
Processing headline 10751: 에스원 최첨단 솔루션 서비스 차별 화로 고객 가치 고
Processing headline 10752: 백신 힘 SK 바사 영업 익 1천억 돌파
Processing headline 10753: 새마을금고 전통 시장 내 범죄예방 환경 설계 사업 지원
Processing headline 10754: SK 쉴더스 IPO 본격 착수 메타 버스 보안 업체 주목 받다 이유 늘다
Processing headline 10755: 하반기 삼성 채용 시 작 올해 도 온라인 입사 비법 전수
Processing headline 10756: 잘나가다 에스원 언 택트 솔루션
Processing headline 10757: fn 마켓 워치 대상 그룹 본사 종로 플 레이스 매물
Processing headline 10758: 클릭 e 종목 에스원 4 Q 기대 이하 실적 목표
Processing headline 10759: 한국 산업 브랜드 파워 끝나다 않다 팬데믹 여파 파워 브랜드 지각 변동
Processing headline 10760: SK 그룹 상장 릴레이 제동 걸

...

Processing headline 15259: 동원 FB 계 열사 공병 설비 양 수 샘물 경쟁력 높이다
Processing headline 15260: 10월 케이 카 NH 투자 증권 42 개사 1억 7296만 주 의무 보유 해제
Processing headline 15261: 마켓 인 ESG 채권 발행 쑥 공사채 쏠리다 아쉽다
Processing headline 15262: 20년 넘다 1 위 지키다 양반 김 원초 다르다
Processing headline 15263: 양 극 박 시장 커지다 알루미늄 업체 잇다 증설
Processing headline 15264: 동 원 시스템 즈 2분 기 영업 익 256억 전 년 동 기비 13.1%
Processing headline 15265: 마켓 인 공기업 아니다 반 기업 도 녹색 채권 발행 쑥
Processing headline 15266: 마켓 뷰 밉다 금리 상승 신흥 국 자금 이탈 ... 코스피 2.8% 하락
Processing headline 15267: 마켓 인사이트 SK 계열 디 앤디 인베스트먼트 지난해 매출 영업 이익 40% 급성 ...
Processing headline 15268: 2020년 의무 보유 주식 전 설정 해제 증가
Processing headline 15269: 동원 vs 하림 승자 늘다 HMM 주가 늘다 약세
Processing headline 15270: 급변 하다 지수 피곤하다 안정 적 인 리츠 주목
Processing headline 15271: 출시 하다 달 만에 100만 계좌 돌파 토스 증권 업계 돌풍 일으키다
Processing headline 15272: 3분 기 주도 업종 금융 에너지 투자 해 라 란 경제 하다 끼
Processing headline 15273: 디 앤디 플랫폼 리츠 내달 유가 증권 시장 상장 추진
Processing headline 15274: 마켓 인사이트 코람코 주택 도 시 기금 리츠 SK DD 플랫폼 리츠 프리 IPO 500억 ...
Processi

Processing headline 15389: 60% 뚝 하다 샘 1분 기 영업 익 100억 주택 거래 원자재
Processing headline 15390: 원자재 인건비 치솟다 침대 소 파의 가구 값 줄 인상
Processing headline 15391: 공간 IoT 전문 기업 고 퀄 우미 건설 20억원 투자 유치
Processing headline 15392: 오늘 집 하다 샘 몸값 뛰어넘다
Processing headline 15393: 먹거리 생필품값 오르다 살기 힘 는데 하반기 더 크다 파도 오다
Processing headline 15394: 초읍 월드메르디앙 에듀 포레 계약 대상 스페셜 기프트 제공
Processing headline 15395: 하다 샘 디자인 가구 데뉴 협업 소파 매트리스 출시
Processing headline 15396: 하다 샘 상반기 시즌 오프 정기 세 진행
Processing headline 15397: 홈쇼핑 강 자도 라 방 켜다 라이브 커머스 경쟁 후끈
Processing headline 15398: 인터뷰 김영호 IMM PE 대표 미샤 눈물 손절 알다 흑자 전환 하다 회사 ...
Processing headline 15399: 스마트 인테리어 플랫폼 데이 개최 교육 세미나 실시
Processing headline 15400: 인수 5년 만에 지다 분 가치 반토막 ... 어피 티 아프다 손가락 되다 락앤락
Processing headline 15401: G 마켓 옥션 집콕족 위 하다 리빙 페어 최대 70% 할인
Processing headline 15402: 하다 샘 유엔 글로벌 컴팩트 가입 ESG 경영 속도
Processing headline 15403: 하다 샘 포 시즌 유미 세포 광고 캠페인 공개
Processing headline 15404: 정책 혜택 겹 치다 가전 시장 위기 에도 환기 가전 힘펠 훨훨
Processing headline 15405: 하다 샘 3 Q 적자 전환 부동산 직격타
Proces

Processing headline 15526: HDC 현대 산업 개발 베스트 파트너 스 데이 개최
Processing headline 15527: HDC 현대 산업 개발 폭염 집중호우 대비 현장 안전 관리 강화
Processing headline 15528: 현산 광주 학동 참사 재개발 사업 시공 계약 유지 종합
Processing headline 15529: 광주 화정 아이파크 계약 납득 가능하다 지다 원안 마련 촉구 종합
Processing headline 15530: HDC 현대 산업 개발 추석 1 주 전 2700억 자금 조기 지급
Processing headline 15531: 9월 지방 3만 6000 가구 공급 추석 연휴 둘러보다 하다 늘다
Processing headline 15532: HDC 현대 산업 개발 주변 지역 유일 순위 내 마감 논산 아이파크 29일 정당 ...
Processing headline 15533: 윤영준 새 주택 협회장 건설업 규제 개선 소통 창구 되다
Processing headline 15534: HDC 현산 안전 품질 쇄신 총력 신뢰 받다 기업
Processing headline 15535: 주변 사다 즐비 아파트 안전 품질 30년 보증
Processing headline 15536: 기업 신용등급 내년 줄줄이 하락 우려
Processing headline 15537: 현산 아파트 붕괴 사고 행정처분 지지 부진 직무 유기 vs 면밀 조사 중
Processing headline 15538: 철산 더 헤리티지 27일 1 순위 청약 접수 ... 선호 도 1 위 브랜드 자다 ...
Processing headline 15539: 박용진 GOS 사태 5만 vs 삼성 노 태문 않다
Processing headline 15540: 매출 늘다 수익 뚝 건설 시련 계절
Processing headline 15541: HDC 현산 19일 화정 아이파크 주거 대책 서류 접수 2630억 지원
Processing headline 15542: 

Processing headline 15664: 고객 퍼스트 전자랜드 가전 유통 채널 우뚝
Processing headline 15665: 연금 브랜드 평판 9월 1 위 미래에셋 자산운용 ... 2 위 3 위 신한은행
Processing headline 15666: 카드 늘다 안되다 보험료 카드 납부 율 10% 대다 불과
Processing headline 15667: A 급 비 우량 채 완판산 은 지원 모회사 신용 보강 월이자지급 식 도입 적중
Processing headline 15668: 롯데 손보 상품 경쟁력 GA 홀렸다
Processing headline 15669: 마켓 인 롯데 손해 보험 후순위채 수 요 예측 서다 790억 모으다
Processing headline 15670: 롯데 손보 4년 만에 보험 영업 재개 메리츠화재 도 확대
Processing headline 15671: 추석 이동 량 증가 보험 손해 율 상승
Processing headline 15672: 인적 분할 앞두다 LG 그룹 하다 달 간 거래 정지
Processing headline 15673: 요즘 자소 서다 카메라 쓰다
Processing headline 15674: 뉴스 BM 혁신 위해 간판 도 바꾸다 회사 이름 바꾸다 기업
Processing headline 15675: 마켓 인사이트 글로벌 로펌 리몬 테크 전문 장 정우 변호사 입 ... 한국 진 ...
Processing headline 15676: fn 마켓 워치 국민연금 석탄 안전 불감증 기업 메스
Processing headline 15677: LG 유 플 3분 기 영업 익 2767억 11년 만에 최대
Processing headline 15678: 니켈 값 급등 배터리 업계 난리 예의 주시 비상 상황 은 알다
Processing headline 15679: 과감 하다 투자 덩치 키우다 LX 그룹
Processing headline 15680: 석탄 실적 효자로 LX 인터내셔널 V 반등
Processing hea

Processing headline 15801: 롯데 렌터카 이용 하다 24시간 무료 이용권 받다
Processing headline 15802: 호텔 신라 CJ 대다 하다 통운 ... 코스피 체급 이동 주목
Processing headline 15803: 차세대 모빌리티 서비스 개척 쏘다 카 롯데 맞다 손
Processing headline 15804: 롯데 렌 탈의 쏘다 카 지다 분 취득 긍정 적 평가 내리다 신평
Processing headline 15805: 외인 손잡다 되다 살아나다 롯데 렌탈
Processing headline 15806: 2월 의무 보유 해제 카뱅 1억 5천만 주크 래 프톤 1천 500만 주
Processing headline 15807: 특징 주 롯데 렌탈 1분 기 호실 적 전망 강세 2분 기도 실적 개선 세 예상 ...
Processing headline 15808: 1년 유예 현대차 기아 기회 삼다 롯데 SK 중고차 업계 투쟁
Processing headline 15809: 이동우 롯데 부회장 바이오 헬 스케 직접 챙기다
Processing headline 15810: 롯데 바이오 직스 달 출범 2030년 글로벌 톱 10 노리다
Processing headline 15811: 신동빈 정용진 통큰 투자 유통 빅 2 5년 간 57조 투입
Processing headline 15812: 모빌리티 첫 유니콘 쏘다 카 폭락 장 뚫다 상장 도전 장
Processing headline 15813: 내 일상 로봇 들어오다 서비스 로봇 시대 개막 하수 정의 티타임
Processing headline 15814: 휴가 철 특수 잡다 카드 프로모션 풍 성
Processing headline 15815: 롯데 렌터카 프리미엄 브랜드 지수 14년 연속 1 위
Processing headline 15816: 공직자 재산 공 개방 기선 기재 1 차관 47억원 신고 대부분 부동산
Processing headline 15817: 단독 쏘다 카 사주 청약 률